<a href="https://colab.research.google.com/github/harshanand63/CodeAlpha_Titanic_Classification/blob/main/harshsign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ultralytics

import os
import cv2
import random
import shutil
from ultralytics import YOLO  # Install using: pip install ultralytics
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Paths and Constants
data_path = "C:/Users/HARSH/Desktop/sign/sign"  # Path to your annotated dataset
output_path = "yolo_dataset"  # Path for YOLO-compatible dataset
os.makedirs(output_path, exist_ok=True)

# Verify dataset path
if not os.path.exists(data_path):
    raise FileNotFoundError(f"The dataset path '{data_path}' does not exist. Please verify the location of your dataset.")

# 1. Data Annotation and Preprocessing
# Assuming you have already annotated the dataset using tools like LabelImg
# Ensure annotations are saved in YOLO format (txt files alongside images).

# 2. Data Augmentation
def augment_images(image_path, save_path):
    """Apply augmentation techniques to images."""
    img = cv2.imread(image_path)

    # Flip
    flipped = cv2.flip(img, 1)
    cv2.imwrite(os.path.join(save_path, f"flip_{os.path.basename(image_path)}"), flipped)

    # Rotation
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, 15, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    cv2.imwrite(os.path.join(save_path, f"rotate_{os.path.basename(image_path)}"), rotated)

# Augment data
images = [f for f in os.listdir(data_path) if f.endswith(".jpg")]
augmented_path = os.path.join(output_path, "augmented")
os.makedirs(augmented_path, exist_ok=True)

for image in tqdm(images, desc="Augmenting Images"):
    augment_images(os.path.join(data_path, image), augmented_path)

# 3. Split Dataset
all_images = [f for f in os.listdir(augmented_path) if f.endswith(".jpg")]
train_files, test_files = train_test_split(all_images, test_size=0.2, random_state=42)

def copy_files(files, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    for file in files:
        shutil.copy(os.path.join(augmented_path, file), dest_folder)

copy_files(train_files, os.path.join(output_path, "train/images"))
copy_files(test_files, os.path.join(output_path, "test/images"))

# Copy corresponding labels
for file_set, dest_folder in zip([train_files, test_files], ["train/labels", "test/labels"]):
    os.makedirs(os.path.join(output_path, dest_folder), exist_ok=True)
    for file in file_set:
        label_file = file.replace(".jpg", ".txt")
        shutil.copy(os.path.join(data_path, label_file), os.path.join(output_path, dest_folder))

# 4. Train the YOLO Model
yolo_model = YOLO("yolov8n.pt")  # YOLOv8n is a lightweight model. Use "yolov8s.pt" for a larger model.

# Train the model
yolo_model.train(
    data=output_path,  # Path to YOLO dataset
    epochs=50,         # Number of epochs
    imgsz=640,         # Image size
    name="signature_detection",  # Name of the project
    project="signature_project"  # Folder to save the results
)

# 5. Evaluate the Model
results = yolo_model.val()
print(results)

# 6. Inference
# Replace 'inference_image.jpg' with your test image
inference_results = yolo_model.predict(source="inference_image.jpg", save=True)
print(inference_results)

# Make sure to zip and upload your project folder to GitHub
print("Training and evaluation completed! Remember to document the steps in README.md.")


FileNotFoundError: The dataset path 'C:/Users/HARSH/Desktop/sign/sign' does not exist. Please verify the location of your dataset.